### Let's play with Interpret ML GBM
---

* It is slow...
* Can be tricky as it currently does not support missing values. 

In [1]:
# !pip install interpret

In [2]:
import os, sys, json
import pickle as pkl
import pandas as pd

from rdsutils import datagen
import rdsutils.plot as rdsplot
from rdsutils.lightgbm_helpers import train_lgb_baseline_grouped
from utils import preprocess

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings(action='ignore')

#### Load data

In [3]:
modeling_df = pd.read_parquet('../../artifacts/final/modeling_df_w_preds.parquet')
test_df = pd.read_parquet('../../artifacts/final/test_df_w_preds.parquet')
modeling_df.shape, test_df.shape

((112599, 241), (152951, 241))

In [4]:
seed = 12345
target_col = 'ach_target'
indeterminate = 'indeterminate_60d'
model_output = 'probability'

modeling_df = preprocess(modeling_df)
test_df = preprocess(test_df)

modeling_df['pred'] = modeling_df['ach_pred_cleaned']
test_df['pred'] = test_df['ach_pred_cleaned']

In [5]:
features = pkl.load(open('../../artifacts/final/features_corr_removed.pkl', 'rb'))
with open(f'../../models/ach-model.pkl', 'rb') as f:
    model = pkl.load(f)

#### Train a simple classifier

In [6]:
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [7]:
ebm = ExplainableBoostingClassifier()

In [8]:
%%time 

ebm.fit(modeling_df[model.feature_name_], modeling_df[target_col])

ValueError: Missing values are currently not supported.

In [9]:
%%time 

ebm_global = ebm.explain_global()
show(ebm_global)

NotFittedError: This ExplainableBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [10]:

ebm_local = ebm.explain_local(test_df[model.feature_name_].head(), 
                              test_df[target_col].head())
show(ebm_local)

NotFittedError: This ExplainableBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
show([logistic_regression, decision_tree])